In [1]:
import re
import pandas as pd
import numpy as np
import json
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import os
from konlpy.tag import Okt

In [2]:
def categoryToTarget(text):
    if text == "판매":
        return 1
    elif text == "교환":
        return 2
    elif text == "구입":
        return 3
    elif text == "거래완료":
        return 4
    elif text == "그냥드림":
        return 5
    else:
        return 100


In [17]:
DATA_IN_PATH = './data/clien/'
fileList = os.listdir(DATA_IN_PATH)

data = pd.DataFrame()
# data = pd.DataFrame(columns=["title","article"])
# targetData = pd.DataFrame(columns=["category"])

for fileName in fileList:
    print(fileName)
    if fileName == ".DS_Store":
        continue
    with open(DATA_IN_PATH + fileName, 'r') as f:
        json_data = json.load(f)
        

    for document in json_data:
        documentDataFrame = pd.DataFrame.from_dict([{
            "title":document['title'],
            "article":document['article'],
            "category":document['category'],
            "category_to_target":categoryToTarget(document['category']),
            "collectDate":document['collectDate'],            
            "registerDate":document['registerDate'],            
        }])
#         articleDataFrame = pd.DataFrame.from_dict([{"title":document['title'],"article":document['article']}])
#         targetDataFrame = pd.DataFrame.from_dict([{"category":categoryToTarget(document['category'])}])

        data = data.append(documentDataFrame)
#         targetData = targetData.append(targetDataFrame)


.DS_Store
20210115_clien
20210116_clien
20210117_clien
20210118_clien
20210119_clien
20210120_clien
20210121_clien
20210122_clien
20210123_clien
20210124_clien
20210125_clien
20210126_clien
20210127_clien
20210128_clien
20210129_clien
20210130_clien
20210131_clien
20210201_clien
20210202_clien
20210203_clien
20210204_clien
20210205_clien
20210206_clien
20210207_clien
20210208_clien
20210209_clien
20210210_clien
20210211_clien
20210212_clien
20210213_clien
20210214_clien
20210215_clien
20210216_clien
20210217_clien
20210218_clien
20210219_clien
20210220_clien
20210221_clien
20210222_clien
20210223_clien
20210224_clien
20210225_clien
20210226_clien
20210227_clien
20210228_clien
20210301_clien
20210302_clien
20210303_clien
20210304_clien
20210305_clien
20210306_clien
20210307_clien
20210308_clien
20210309_clien
20210310_clien
20210311_clien
20210312_clien
20210313_clien
20210314_clien
20210315_clien
20210316_clien
20210317_clien
20210318_clien
20210319_clien
20210320_clien
20210321_clien


In [18]:
okt=Okt()
article_text = okt.morphs(data.iloc[0]['article'])
# print(article_text)
stopWords = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']

def preprocessing(text, okt, remove_stopwords= False, stop_words=[]):
    #줄바꿈 문자 삭제
    text = text.replace("\n","")
    wordText = okt.morphs(text, stem=True)
    
    if remove_stopwords:
        wordText = [token for token in wordText if not token in stop_words]
        
    return wordText

In [ ]:
cleanTrainArticle = [preprocessing(article, okt, remove_stopwords=True, stop_words=stopWords) for article in data['article']]

In [12]:
len(cleanTrainArticle)

cleanTrainDF = pd.DataFrame({'article':cleanTrainArticle, 'category':data['category_to_target']})
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleanTrainArticle)
text_sequences = tokenizer.texts_to_sequences(cleanTrainArticle)

print(text_sequences[0:5])
wordVocab = tokenizer.word_index
wordVocab["<PAD>"] = 0
# print(wordVocab)
# print(len(wordVocab))

[[149, 37, 79, 2616, 393, 691, 340, 11, 2, 1, 97, 600, 195, 9, 1, 946, 175, 947, 691, 340, 4, 1, 3170, 120, 1, 1300, 10, 1301, 4, 1], [20, 1, 1195, 133, 885, 394, 7, 1625, 8, 332, 13, 601, 629, 6, 508, 2167, 1849, 1850, 44, 1625, 446, 1851, 6, 27, 2, 97, 257, 68, 1, 5, 42, 72, 73, 9, 748, 37, 36, 9, 748, 37, 60, 6, 1302, 33, 1303, 9, 749, 55, 2617, 68, 1852, 332, 42, 72, 35, 179, 172, 1, 35, 283, 24, 1, 17, 1, 1195, 1626, 20, 1627, 5, 13, 446, 1628, 27, 2, 1, 5, 1629, 434, 1630, 290, 42, 72, 1304, 55, 17, 1853, 948, 296, 36, 97, 257, 68, 1, 2618, 1305, 1429, 101, 259, 425, 9, 1, 354, 1854, 6, 11, 2, 1, 1306, 552, 12, 11, 107, 297, 1, 886, 17, 949, 3, 630, 20, 140, 1855, 1856, 26, 21, 19, 1, 14, 41, 364, 655, 298, 22, 435, 19, 1, 1300, 10, 1301], [110, 570, 20, 125, 11, 2, 1, 27, 52, 16, 509, 32, 11, 2, 37, 201, 4, 1857, 25, 77, 4538, 16, 2, 43, 1098, 25, 1631, 3171, 97, 32, 7, 35, 179, 8, 21, 3, 14, 2, 1], [207, 186, 17, 57, 5, 409, 6, 75, 11, 2, 244, 28, 124, 49, 87, 2, 602, 1632, 12,

In [15]:
data_configs = {}
data_configs['vocab'] = wordVocab
data_configs['vocab_size'] = len(wordVocab)+1

MAX_SEQUENCE_LENGTH = 200
train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
print(train_inputs.shape)
train_label = np.array(data['category_to_target'])
print(train_label.shape)

(1181, 200)
(1181,)


In [16]:
DATA_IN_PATH = './clean_data/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS = 'data_configs.json'

if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_label)
cleanTrainDF.to_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, index=False)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)